In [1]:
import torch

pretrained_weights  = torch.load('./models/libra_faster_rcnn_x101_64x4d_fpn_1x_20190525-359c134a.pth')

num_class = 2
pretrained_weights['state_dict']['bbox_head.fc_cls.weight'].resize_(num_class, 1024)
pretrained_weights['state_dict']['bbox_head.fc_cls.bias'].resize_(num_class)
pretrained_weights['state_dict']['bbox_head.fc_reg.weight'].resize_(num_class*4, 1024)
pretrained_weights['state_dict']['bbox_head.fc_reg.bias'].resize_(num_class*4)

torch.save(pretrained_weights, "./models/libra_faster_rcnn_x101_64x4d_fpn_1x_%d.pth"%num_class)

In [1]:
from __future__ import division
import argparse
import os

import torch
from mmcv import Config

from mmdet import __version__
from mmdet.apis import (get_root_logger, init_dist, set_random_seed,
                        train_detector)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

In [2]:
config_file = 'configs/01_faster_rcnn_r50_fpn_1x-SAU.py'
work_dir='work_dirs/a01_20191021'
#checkpoint_file = 'checkpoints/faster_rcnn_r50_fpn_1x_20181010-3d1b3351.pth'
gpus = 2

In [3]:
## RUN in cmd to get mAP and validation. Distributed!

./tools/dist_train.sh ${CONFIG_FILE} ${GPU_NUM} [optional arguments]

--validate (strongly recommended): Perform evaluation at every k (default value is 1, which can be modified like this) epochs during the training.
--work_dir ${WORK_DIR}: Override the working directory specified in the config file.
--resume_from ${CHECKPOINT_FILE}: Resume from a previous checkpoint file.
   
./tools/dist_train.sh 'configs/05_faster_rcnn_r50_fpn_1x-SAU.py' 2 --validate --work_dir 'work_dirs/20191022_01'

/home/thaikari/anaconda3/bin/python: Error while finding module specification for 'torch.distributed.launch' (ModuleNotFoundError: No module named 'torch')


In [3]:
#THIS fails to launch distributed...
def train(config,
          gpus,
          autoscale_lr=False,
          launcher='pytorch',
          local_rank=0,
          resume_from=None,
          seed=None,
          validate=False,
          work_dir=None):
    
    
    cfg = Config.fromfile(config_file)
    # set cudnn_benchmark
    if cfg.get('cudnn_benchmark', False):
        torch.backends.cudnn.benchmark = True
    # update configs according to CLI args
    if work_dir is not None:
        cfg.work_dir = work_dir
    if resume_from is not None:
        cfg.resume_from = resume_from
    cfg.gpus = gpus

    if autoscale_lr:
        # apply the linear scaling rule (https://arxiv.org/abs/1706.02677)
        cfg.optimizer['lr'] = cfg.optimizer['lr'] * cfg.gpus / 8

    # init distributed env first, since logger depends on the dist info.
    if launcher == 'none':
        distributed = False
    else:
        distributed = True
        init_dist(launcher, **cfg.dist_params)

    # init logger before other steps
    logger = get_root_logger(cfg.log_level)
    logger.info('Distributed training: {}'.format(distributed))

    # set random seeds
    if seed is not None:
        logger.info('Set random seed to {}'.format(seed))
        set_random_seed(seed)

    model = build_detector(
        cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

    datasets = [build_dataset(cfg.data.train)]
    if len(cfg.workflow) == 2:
        datasets.append(build_dataset(cfg.data.val))
    if cfg.checkpoint_config is not None:
        # save mmdet version, config file content and class names in
        # checkpoints as meta data
        cfg.checkpoint_config.meta = dict(
            mmdet_version=__version__,
            config=cfg.text,
            CLASSES=datasets[0].CLASSES)
    # add an attribute for visualization convenience
    model.CLASSES = datasets[0].CLASSES
    print('=====================')
    print('VALIDATE', validate)
    train_detector(
        model,
        datasets,
        cfg,
        distributed=distributed,
        validate=validate,
        logger=logger)

In [4]:
train(config_file, gpus, work_dir='work_dirs/a00_20191021', validate=True)

KeyError: 'RANK'

In [ ]:
##IN config file:

#Change lr => 0.0005 from 0.02 because 2gpus instead of 8 gpus
#start_from = checkpoint_file
#frozen_stages = 3 (from 1)

In [ ]:


#for param in model.parameters(): # Freeze all parameters\n",
#    param.requires_grad = False
    
#for param in model.fc.parameters(): # Unfreeze the last fully - connected
#    print(param)

In [ ]:
cfg = Config.fromfile(config_file)
datasets = [build_dataset(cfg.data.train)]

datasets[0].CLASSES


In [ ]:
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
